Create an annotated calendar plot using the create_calendar function. 

In this example gist, hue of the cell denotes the traffic observed in the store while the text in each cell denotes the sales. 

st_date, end_date are datetime.date objects that can be used to filter the data that needs to be plotted. 


In [1]:
import os
import numpy as np
import pandas as pd
import datetime

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

In [2]:
def get_dummy_data(n, st_date = datetime.date(2019, 1, 1)):
    
    df = pd.DataFrame({'date':[st_date + datetime.timedelta(i) for i in range(n)],
                       'traffic':np.random.normal(100, 50, n).astype(int), 
                       'sales': np.random.normal(1000, 500, n).astype(int)})
    
    return df


In [3]:
def create_calendar(df, st_date, end_date, text_var='sales', hue_var='traffic'):
    
        
    df = df[(df['date']>=st_date) & (df['date']<=end_date)].copy().reset_index(drop=True)
    df['dow'] = [x.weekday() for x in df['date']]
    
    
    start_dow = int(df.loc[0, 'dow'])
    last_dow = int(df['dow'].iloc[-1])
    last_date = df['date'].iloc[-1]
    
    # Insert elements in first row to ensure that dates start from Monday
    j = 0
    for i in range(start_dow):
        df = df.append({'dow':j, 'date':df.date.iloc[0] + datetime.timedelta(-i-1)}, 
                       ignore_index=True, sort=True)
        j = j+1
        
    # Insert elements in last row till last date is a Sunday
    if(last_dow !=6):
        j = 1
        for i in range(last_dow+1, 7):
            df = df.append({'date':last_date + datetime.timedelta(j), 'ht_idx':0}, 
                           ignore_index=True, sort=True)
            j = j+1
    
    df['date_str'] = [str(x) for x in df['date']]
    df = df.sort_values(by='date_str').reset_index(drop=True)
    df['dow'] = [x.weekday() for x in df['date']]              # gives x-axis 
    df['ht_idx'] = [-1*(i//7) for i in range(len(df))]         # gives y-axis
    df['date_fmt'] = [d.ctime()[4:10] for d in df['date']]
    
       
    dates = go.Scatter(
            x=df.dow,
            y=df.ht_idx,
            mode='text',
            name='Dates',
            text=df.date_fmt,
            textfont=dict(size=16, family='Courier New, monospace'),
            textposition='top center')

    trace1 = go.Heatmap(x=df['dow'], y=df['ht_idx'], z=df[hue_var], xgap=2, ygap=2, 
                        opacity=0.6, hoverinfo='skip',
                        colorscale=[[0, 'rgb(166,206,227)'], [0.2, 'rgb(31,120,180)'], 
                                   [0.4, 'rgb(178,223,138)'], [0.6, 'rgb(51,160,44)'], 
                                   [0.8, 'rgb(251,154,153)'], [1, 'rgb(227,26,28)']])

    trace2 = go.Scatter(
            x=df.dow,
            y=df.ht_idx,
            mode='text',
            name='Sales',
            text=df[text_var],
            textposition='bottom center')
    
    layout = go.Layout(title=f'Calender view of sales', showlegend=False, yaxis=dict(autorange=True, 
                             showgrid=False, zeroline=False, showline=False, ticks='', showticklabels=False),
                       xaxis=dict(title="Day of the week",  showgrid=False, showline=False, zeroline=False, 
                             tickvals=[0, 1, 2, 3, 4, 5, 6], ticktext=['M', 'T', "W","Th","F","Sa","Su"]))

    iplot({"data":[dates, trace1, trace2], "layout":layout})
    

## Create Annotated Calendar Plot

In [4]:
df = get_dummy_data(n=60, st_date=datetime.date(2019, 1, 1))

print(df.head())

         date  traffic  sales
0  2019-01-01      116    582
1  2019-01-02       64   1928
2  2019-01-03       42    424
3  2019-01-04      145    530
4  2019-01-05       64   2082


In [5]:
st_date = datetime.date(2019, 1, 15)
end_date = datetime.date(2019, 2, 15)
create_calendar(df, st_date, end_date, text_var='sales', hue_var='traffic')

![title](calender.png)

Dates not falling between st_date and end_date are shown with no values